# **Import Required Libraries**

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

# **Loading and Pre Processing the Data**

In [ ]:
train_labels = pd.read_csv('../input/game-of-deep-learning-ship-datasets/train/train.csv')
train_labels['category'] = train_labels['category'] - 1
train_labels.head()

In [ ]:
X = []
y = []
path = "../input/game-of-deep-learning-ship-datasets/train/images/"
img_shape = 128
for i in train_labels.iterrows() :
    img_path = os.path.join(path, i[1]['image'])
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_shape,img_shape), interpolation='lanczos')
    img = tf.keras.preprocessing.image.img_to_array(img)
    X.append(img)
    y.append(float(i[1]['category']))
X = np.asarray(X)
y = np.asarray(y)
X.shape, y.shape

In [ ]:
X = X / 255.0

# **Viewing Sample Images from the Dataset**

In [ ]:
plt.imshow(X[1000])
plt.title(y[1000])

# **Creating a Deep Learning model using Keras**

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((img_shape,img_shape,3)),
    tf.keras.applications.MobileNet(include_top=False, weights='imagenet'),
    tf.keras.layers.Conv2D(3, 3, padding='same', activation='sigmoid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4), metrics=['accuracy'])

model.summary()

# **Training the Deep Learning Model**

In [ ]:
model.fit(X, y, epochs=50, batch_size=128, validation_split=0.25, shuffle=True)

# **Evaluating the model on whole dataset**

In [ ]:
model.evaluate(X,y)

# **Defining labels for prediction**

In [ ]:
labels_inv = {'Cargo': 0,
              'Military': 1,
              'Carrier': 2,
              'Cruise': 3,
              'Tankers': 4}
labels = {v:k for k,v in labels_inv.items()}
labels

# **Helper Function for Prediction**

In [ ]:
def predict(path) :
    img = tf.keras.preprocessing.image.load_img(path, target_size=(img_shape,img_shape), interpolation='lanczos')
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0
    pred = model.predict(np.array([img]))
    plt.imshow(img)
    pred_label = np.argsort(pred)
    for i in pred_label[0][-1:-3:-1] :
        print(f"{labels[i]} Ship : {pred[0][i]*100:0.2f} %")

# **Testing the model on various data obtained from Intrnet**

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/1/1c/MAERSK_MC_KINNEY_M%C3%96LLER_%26_MARSEILLE_MAERSK_%2848694054418%29.jpg/1200px-MAERSK_MC_KINNEY_M%C3%96LLER_%26_MARSEILLE_MAERSK_%2848694054418%29.jpg

In [ ]:
predict("./1200px-MAERSK_MC_KINNEY_MÖLLER_&_MARSEILLE_MAERSK_(48694054418).jpg") # Cargo Ship Example from Internet

In [ ]:
!wget https://media.architecturaldigest.com/photos/593829315838160353567328/master/pass/mscworldclass.jpg

In [ ]:
predict("./mscworldclass.jpg") # Cruise Ship Example from Internet

In [ ]:
!wget https://images.livemint.com/img/2020/07/06/600x338/USaircraftcarriers-k59B--621x414@LiveMint_1594024838759.jpg

In [ ]:
predict("./USaircraftcarriers-k59B--621x414@LiveMint_1594024838759.jpg") # Military Ship Example from Internet

In [ ]:
!wget https://www.dnv.com/Images/KP_Tan_269_Tanker_vessel_tcm71-192023.jpg

In [ ]:
predict("./KP_Tan_269_Tanker_vessel_tcm71-192023.jpg") # Tanker Ship Example from Internet

In [ ]:
!wget https://images02.military.com/sites/default/files/2020-02/uss-independence-eastern-pacific-1800.jpg

In [ ]:
predict("./uss-independence-eastern-pacific-1800.jpg")

In [ ]:
!wget https://cdn.britannica.com/34/94834-050-16DB7EEB/USS-Carl-Vinson-aircraft-carrier-US-Navy-2005.jpg

In [ ]:
predict("./USS-Carl-Vinson-aircraft-carrier-US-Navy-2005.jpg")

In [ ]:
!wget https://chantiers-atlantique.com/wp-content/uploads/2021/05/Page-Activite%CC%81s-BPC-N33-G-A-NASSER-25-5-1-min-1980x840.jpg

In [ ]:
predict("./Page-Activités-BPC-N33-G-A-NASSER-25-5-1-min-1980x840.jpg")

In [ ]:
!wget https://img.theweek.in/content/dam/week/news/india/images/2020/2/19/kavaratti-grse.jpg

In [ ]:
predict("./kavaratti-grse.jpg")

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/7/75/INS_Mumbai_%28D62%29_underway.jpg

In [ ]:
predict("./INS_Mumbai_(D62)_underway.jpg")

In [ ]:
!wget https://www.theenterpriseworld.com/wp-content/uploads/2021/03/suez-canal-ship-stuck.jpg

In [ ]:
predict("./suez-canal-ship-stuck.jpg")

# **Codes for getting Submission File for the Competition**

In [ ]:
sub = pd.read_csv("../input/game-of-deep-learning-ship-datasets/sample_submission_ns2btKE.csv")
sub.head()

In [ ]:
X_sub = []
path = "../input/game-of-deep-learning-ship-datasets/train/images/"
img_shape = 128
for i in sub.iterrows() :
    img_path = os.path.join(path, i[1]['image'])
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_shape,img_shape), interpolation='lanczos')
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0
    X_sub.append(img)
X_sub = np.asarray(X_sub)
X_sub.shape

In [ ]:
sub.shape

In [ ]:
y_sub_cat = model.predict(X_sub)
y_sub = np.argmax(y_sub_cat, axis=1)
y_sub.shape

In [ ]:
sub['category'] = y_sub + 1

In [ ]:
sub['category'].value_counts()

In [ ]:
sub.to_csv('submission.csv', index=False)

In [ ]:
import IPython.display as p
p.FileLink('./submission.csv')